In [6]:
from owlready2 import *
import shutil

AttributeError: module 'owlready2' has no attribute '__version__'

In [2]:
from util import file_util
from util.graph_util import Graph
from service import python_content_creator

In [3]:
member_propagation = {}
cp_map = {}

template = """
{import_statements}\n\n
class {class_name}({parent}):
    {functions}
"""

def generate_imports_from_template(node):
    import_template = """\n{lib}\n{core}"""
    core = node.core_import
    lib = node.lib_import
    return import_template.format(
        lib=node.lib_import.first() if lib else "",
        core=node.core_import.first() if core else "")
    
def generate_class_from_template(node, parent, functions):
    return template.format(
        import_statements=generate_imports_from_template(node), 
        class_name=node.label.first(), 
        parent= parent.label.first() if parent else "", 
        functions=functions)

def function_name_handler(func_name):
    if func_name == "init":
        return "__init__"
    return func_name
        
def generate_model_init_from_template(node, params, variables):
    if node.core_import:
        lib_name = node.core_import.first().split()[-1]
        variables.extend(params.split(", "))
        variables = set(variables)
        stmts = ["{var} = self.{var}".format(var=var) for var in variables]
        func_params = ",\n\t\t\t".join(stmts)
        return "\n\t\tself.model = {lib_name}({func_params})".format(
            lib_name=lib_name, func_params=func_params)
    return ""

def generate_function_body_from_template(node, func, target):
    print("FUNC: ", func.label.first())
    func_name = func.label.first()
    if func_name == "init":
        variables = [obj.label.first() for obj in target]
        stmts = ["self.{var} = {var}".format(var=var) for var in variables]
        stmt = "\n\t\t".join(stmts)
        params = get_inherited_params(node)
        if params:
            stmt = stmt + "\n\t\t{parent}.__init__(self, {params})".format(
                parent=cp_map[node].label.first(), params=params)
        stmt = stmt + generate_model_init_from_template(node, params, variables)
        return stmt
    else:
        variables = [obj.label.first() for obj in target]
        stmts = ["{var}={var}".format(var=var) for var in variables]
        params = ",\n\t\t\t".join(stmts)
        return "return self.model.{func_name}({params})".format(
            func_name=func_name,
            params=params
        )
        

def generate_function_param_from_template(node, target):
    params = ""
    param_template = "{var} = {value}"
    for obj in target:
        if not obj.default:
            param = obj.label.first()
        else:
            param = param_template.format(
                var=obj.label.first(), value=obj.default.first())
        params = params + ", " + param
    inherited_params = get_inherited_params(node)
    if inherited_params:
        params = ", " + inherited_params + params 
    return params

def generate_function_from_template(node, func):
    func_template = """
    def {func_name}(self{params}):
        {statements}
    
    """
    print("\n---------------------")
    print("LABEL: ",func.label[0])
    var = func.label.first()
    target = eval("node."+var)
    print("TARGET: ",target)
    if var == "init":
        for child in node.descendants():
            if child != node:
                if member_propagation.get(child):
                    member_propagation[child] = member_propagation[child] + target
                else:
                    member_propagation[child] = target
    if target:
        func = func_template.format(
            func_name=function_name_handler(func.label.first()),
            params=generate_function_param_from_template(node, target),
            statements=generate_function_body_from_template(node, func, target)
        )
        return func

def get_inherited_params(node):
    inherited_variables = member_propagation.get(node)
    print("INH_P: ", inherited_variables)
    if inherited_variables:
        var = [obj.label.first() for obj in inherited_variables]
        params = ", ".join(var)
        return params
    return ""
    
def generate_init_by_member_propagation(node):
    func_template = """
    def __init__(self, {params}):
        {parent}.__init__(self, {params}){stmt}
    """
    params = get_inherited_params(node)
    return func_template.format(
        params=params,
        parent=cp_map[node].label.first(),
        stmt=generate_model_init_from_template(node, params, [])
    )
    
    
def generate_functions_from_template(node):
    function_names = set()
    func_data = ""
    for func in node.get_class_properties():
        if func.label:
            func_data = func_data + generate_function_from_template(node, func)
            function_names.add(func.label[0])
    if not func_data and member_propagation.get(node):
            func_data = generate_init_by_member_propagation(node)
    return func_data
        

def create_file_contents(node, child_parent_map):
    print(node.label)
    global cp_map
    cp_map = child_parent_map
    parent = cp_map[node]
    if parent:
        print("PARENT: ", parent)
    func_data = generate_functions_from_template(node)
    if not func_data:
        func_data = "pass"
    content = generate_class_from_template(node, parent, func_data)
    print(content)
    return content
    #print("MEM_PROP: ", member_propagation)

In [4]:
import os

child_parent_map = {}
onto = get_ontology("ml_algorithms.owl").load()
queue = []
root_class = None
print([i for i in onto.annotation_properties()])
print(list(onto.classes()))
for onto_class in list(onto.classes()):
    print(onto_class)
    print(onto_class.label)
    if(onto_class.label[0] == 'MLalgorithms'):
        root_class = onto_class

queue.append(root_class)
dir_structure = [root_class.label[0]]
file_path = []
child_parent_map[root_class] = None
while queue:
    node = queue.pop(0)
    file = dir_structure.pop(0)
    if onto.get_children_of(node):
        file_util.create_folders_and_subfolders(file)
    content = create_file_contents(node, child_parent_map)
    file_util.create_and_write_file(file + ".py", content)
    file_path.append(file)
    for child in onto.get_children_of(node):
        queue.append(child)
        child_parent_map[child] = node
        dir_structure.append(os.path.join(file, child.label[0]))

del member_propagation
del cp_map
print(file_path)

[ml_algorithms.core_import, ml_algorithms.default, ml_algorithms.import, ml_algorithms.lib_import, ml_algorithms.mla:default, ml_algorithms.mla:dtype]
[ml_algorithms.Bayesian, ml_algorithms.MLalgoritthms, ml_algorithms.Classification, ml_algorithms.Clustering, ml_algorithms.GRU, ml_algorithms.RNNBase, ml_algorithms.LSTM, ml_algorithms.LassoRegression, ml_algorithms.Regression, ml_algorithms.max_iter, ml_algorithms.LassoTempReg, ml_algorithms.alpha, ml_algorithms.LinearRegression, ml_algorithms.NeuralNetwork, ml_algorithms.RNN, ml_algorithms.nonlinearity, ml_algorithms.batch_first, ml_algorithms.bias, ml_algorithms.bidirectional, ml_algorithms.dropout, ml_algorithms.hidden_size, ml_algorithms.input_size, ml_algorithms.num_layers, ml_algorithms.X, ml_algorithms.sample_weight, ml_algorithms.y, ml_algorithms.copy_X, ml_algorithms.fit_intercept, ml_algorithms.n_jobs, ml_algorithms.normalize, ml_algorithms.RidgeRegression, ml_algorithms.parameters, ml_algorithms.attributes, ml_algorithms.coe

In [ ]:
for i in list(root_class.descendants()):
    if i.label[0] == 'Regression':
        for j in list(i.descendants()):
            print(j.label[0])
            for k in list(onto.object_properties()):
                try:
                    print("df",k)
                except:
                    pass

In [72]:
class ML:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def fit(self, X, y):
        self._model.printf(X)

In [73]:
class student:
    def printf(self, X):
        print(X)

In [76]:
class DT(ML):
    def __init__(self, d, e):
        self.x = d
        self.y = e
        ML.__init__(self, d,e)
        self._model = student()
    
    def printl(self):
        print(self.b)

In [77]:
tree = DT(5,4)
tree.fit(5,4)
tree.printl()
tree._model.printf(5)

5
4
5


In [18]:
class A:
    def __init__(self, name):
        self.name = name
        print("A")

class B(A):
    def __init__(self, name):
        print("B")
        A.__init__(self, name=name)

In [16]:
class C(B):
    def __init_(self, a, b):
        self.b = b
        print("C")
        B.__init__(self, a)

In [19]:
c = B("varun")

B
A


In [20]:
c.name

'varun'

In [52]:
target = [1,2,3,4]

In [53]:
d = {}

In [54]:
d["A"] = [1]

In [55]:
d["A"].extend(target)

In [41]:
d["A"] = d["A"].append(target)

In [44]:
d["A"].append(target)

In [71]:
a =  True
b = False

In [79]:
for i in None:
    print("HI")

TypeError: 'NoneType' object is not iterable

In [15]:
test_str = "from sklearn.linear_model import Lasso"

In [19]:
test_str.split()[-1]

'Lasso'

In [6]:
temp = {'obj1': [1,2,3], 'obj2': [1,2,3], 'obj3': [1,2,3]}

In [7]:
child='obj1'
temp[child].extend([5])

In [11]:
[6,7] + [8]

[6, 7, 8]

{'obj1': [6, 7], 'obj2': [1, 2, 3], 'obj3': [1, 2, 3]}